In [ ]:
import torch
from flask import Flask, request, jsonify, render_template
import ML
from TFRS import net, train_iter, test_iter, device, model_path, labels, test_transform

In [ ]:
# 训练
lr, num_epochs = 0.01, 3
ML.image_classification_train(net, train_iter, test_iter, num_epochs, lr, device)
torch.save(net.state_dict(), model_path)

In [ ]:
# 测试
net.load_state_dict(torch.load(model_path, map_location=device))
ML.image_classification_test(net, labels, test_iter, device)
print(ML.evaluate_accuracy(net, test_iter, device))

In [ ]:
# 预测
app = Flask(__name__)
net.load_state_dict(torch.load(model_path, map_location=device))
net.to(device)


@app.route('/', methods=['GET'])
def home():
    return render_template('index.html')


@app.route('/predict', methods=['POST'])
def predict():
    image = request.files['image']
    predicted_class = ML.image_classification_predict(net, labels, image, test_transform, device)
    response = jsonify({'predicted_class': predicted_class})
    response.headers['Access-Control-Allow-Origin'] = '*'
    return response


app.run()